# DataMining TwitterAPI

Requirements:
- TwitterAccount
- TwitterApp credentials

## Imports
The following imports are requiered to mine data from Twitter

In [ ]:
# http://tweepy.readthedocs.io/en/v3.5.0/index.html
import tweepy
# https://api.mongodb.com/python/current/
import pymongo
import json
import sys

## Access and Test the TwitterAPI
Insert your `CONSUMER_KEY`, `CONSUMER_SECRET`, `ACCESS_TOKEN` and `ACCESS_TOKEN_SECRET` and run the code snippet to test if access is granted. If everything works well 'tweepy...' will be posted to your timeline.

In [ ]:
# Set the received credentials for your recently created TwitterAPI
CONSUMER_KEY = 'MmiELrtF7fSp3vptCID8jKril'
CONSUMER_SECRET = 'HqtMRk4jpt30uwDOLz30jHqZm6TPN6rj3oHFaL6xFxw2k0GkDC'
ACCESS_TOKEN = '116725830-rkT63AILxR4fpf4kUXd8xJoOcHTsGkKUOKSMpMJQ'
ACCESS_TOKEN_SECRET = 'eKzxfku4GdYu1wWcMr5iusTmhFT35cDWezMU2Olr5UD4i'

# auth with your provided 
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# Create an instance for the TwitterApi
twitter = tweepy.API(auth)
status = twitter.update_status('tweepy ...')
print(json.dumps(status._json, indent=1))

## mongoDB
To gain access to the mongoDB the library `pymongo` is used.

In the first step the mongoDB URL is defined.

In [ ]:
MONGO_URL = 'mongodb://twitter-mongodb:27017/'

Next, two functions are defined to save and load data from mongoDB.

In [ ]:
def save_to_mongo(data, mongo_db, mongo_db_coll):
    # Connects to the MongoDB server running on
    client = pymongo.MongoClient(MONGO_URL)
    # Get a reference to a particular database
    db = client[mongo_db]
    # Reference a particular collection in the database
    coll = db[mongo_db_coll]
    # Perform a bulk insert and return the IDs
    return coll.insert_one(data)

def load_from_mongo(mongo_db, mongo_db_coll, return_cursor=False, criteria=None, projection=None):
    # Optionally, use criteria and projection to limit the data that is
    # returned - http://docs.mongodb.org/manual/reference/method/db.collection.find/
    
    # Connects to the MongoDB server running on
    client = pymongo.MongoClient(MONGO_URL)
    # Reference a particular collection in the database
    db = client[mongo_db]
    # Perform a bulk insert and return the IDs
    coll = db[mongo_db_coll]
    if criteria is None:
        criteria = {}
    if projection is None:
        cursor = coll.find(criteria)
    else:
        cursor = coll.find(criteria, projection)
    
    # Returning a cursor is recommended for large amounts of data
    if return_cursor:
        return cursor
    else:
        return [ item for item in cursor ]

## Stream tweets to mongoDB
Now we want to stream tweets to a current trend to the mongoDB.

Therefore we ask the TwitterAPI for current Trends within different places. Places are defined with WOEID https://www.flickr.com/places/info/1

In [ ]:
# WORLD
print('trends WORLD')
trends = twitter.trends_place(1)[0]['trends']
for t in trends[:5]:
    print(json.dumps(t['name'],indent=1))
# US
print('\ntrends US')
trends = twitter.trends_place(23424977)[0]['trends']
for t in trends[:5]:
    print(json.dumps(t['name'],indent=1))
# AT
print('\ntrends AUSTRIA')
trends = twitter.trends_place(23424750)[0]['trends']
for t in trends[:5]:
    print(json.dumps(t['name'],indent=1))

### StreamListener
tweepy provides a StreamListener that allows to stream live tweets. All streamed tweets are stored to the mongoDB.

In [ ]:
MONGO_DB = 'trends'
MONGO_COLL = 'tweets'

TREND = '#BestBoyBand'

class CustomStreamListener(tweepy.StreamListener):
    def __init__(self, twitter):
        self.twitter = twitter
        super(tweepy.StreamListener, self).__init__()
        self.db = pymongo.MongoClient(MONGO_URL)[MONGO_DB]
        self.number = 1
        print('Streaming tweets to mongo ...')

    def on_data(self, tweet):
        self.number += 1
        self.db[MONGO_COLL].insert_one(json.loads(tweet))
        if self.number % 100 == 0 : print('{} tweets added'.format(self.number))

    def on_error(self, status_code):
        return True # Don't kill the stream

    def on_timeout(self):
        return True # Don't kill the stream

sapi = tweepy.streaming.Stream(auth, CustomStreamListener(twitter))
sapi.filter(track=[TREND])

## Collect tweets from a specific user
In this use-case we mine data from a specific user.

In [ ]:
MONGO_DB = 'trump'
MONGO_COLL = 'tweets'

TWITTER_USER = '@realDonaldTrump'

def get_all_tweets(screen_name):
    #initialize a list to hold all the tweepy Tweets
    alltweets = []    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = twitter.user_timeline(screen_name = screen_name,count=200)
    #save most recent tweets
    alltweets.extend(new_tweets)
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = twitter.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        #save most recent tweets
        alltweets.extend(new_tweets)
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        print("...{} tweets downloaded so far".format(len(alltweets)))
       
    #write tweet objects to JSON 
    print("Writing tweet objects to MongoDB please wait...")
    number = 1
    for status in alltweets:
        print(save_to_mongo(status._json, MONGO_DB, MONGO_COLL))
        number += 1
    print("Done - {} tweets saved!".format(number))

#pass in the username of the account you want to download
get_all_tweets(TWITTER_USER)

## Load tweets from mongo

In [ ]:
data = load_from_mongo('trends', 'tweets')
for d in data[:5]:
    print(d['text'])